# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [1]:
import requests
import pandas as pd

## Список объектов



In [2]:
offset_ = 2001
limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

In [3]:
objects_data.keys()

dict_keys(['data', 'errcode'])

In [4]:
objects_list = objects_data.get('data').get('list')

In [5]:
objids = [x.get('objId') for x in objects_list]

In [6]:
len(objids)

1000

In [7]:
objids[:10]

[38724, 36637, 12856, 44876, 39480, 42036, 42038, 36633, 34617, 28916]

# Данные по определенному объекту

In [8]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/25002'

In [9]:
res = requests.get(url)
res.json()

{'data': {'id': 25002,
  'pdId': 63825,
  'developer': {'devId': 2666,
   'devShortCleanNm': '10 КВАРТАЛ',
   'devShortNm': '10 КВАРТАЛ',
   'devFullCleanNm': '10 КВАРТАЛ',
   'problObjCnt': 0,
   'buildObjCnt': 0,
   'comissObjCnt': 19,
   'regRegionDesc': 'Московская область',
   'devPhoneNum': '+7(495)107-91-24',
   'devSite': 'www.mechta.su',
   'devEmail': 'resnov@cmft.ru',
   'devInn': '5050102110',
   'devOgrn': '1135050000447',
   'devKpp': '500701001',
   'devLegalAddr': 'Московская область, город Дмитров, улица Весенняя дом 29 корпус - строение - комната 2',
   'devFactAddr': 'обл Московская Дмитровский город Дмитров село Озерецкое д. 29 комната 2 50',
   'fundGuarantyFlg': 0,
   'devOrgRegRegionCd': 50,
   'devEmplMainFullNm': 'Меламуд Александр Романович',
   'orgForm': {'id': 4,
    'fullForm': 'Общество с ограниченной ответственностью',
    'shortForm': 'ООО'},
   'objGuarantyEscrowFlg': 0,
   'govFundFlg': 0},
  'region': 50,
  'address': 'село Озерецкое, д. 30',
  'name

## Задание 1

- Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
- Cохранить ее в pandas dataframe
- Cохранить pandas dataframe в excel
- Cохранить pandas dataframe в pickle
- Cохранить pandas dataframe в БД

In [10]:
offset = 0       
limit = 1000   
objects_ids_all = [] 

while True:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    data = res.json()['data']
    data_length = len(data['list'])
    if data_length > 0:
        objects_list = data['list']
        objects_ids = [x.get('objId') for x in objects_list]
        objects_ids_all += objects_ids
        offset += limit
    else:
        break 
        
len(objects_ids_all)

9953

In [12]:
df = []

for id in objects_ids_all:
    cell = {}
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    res = requests.get(url)
    data = res.json()['data']
    try:
        cell['id'] = data.get('id')
        cell['region'] = data.get('region')
        cell['address'] = data.get('address')
        cell['name'] = data.get('nameObj')
        cell['floors'] = data.get('floorMin')
        cell['flats'] = data.get('objElemLivingCnt')
        cell['parking'] = data.get('objElemParkingCnt')
        cell['ready_date'] = data.get('objReady100PercDt')
        cell['material'] = data.get('wallMaterialShortDesc')
        cell['price'] = data.get('objPriceAvg')
        cell['sqare'] = data.get('objFlatSq')
        cell['class'] = data.get('objLkClassDesc')
        cell['url'] = url
    except:
        break
    df.append(cell)
    
df = pd.DataFrame(df) # создаем датафрейм из таблицы
df.set_index('id', inplace=True)
df.head()

,region,address,name,floors,flats,parking,ready_date,material,price,sqare,class,url
id,,,,,,,,,,,,
13391,72,"г Тюмень, ул Профсоюзная",None,3,44,41,2022-06-30,Другое,55893.0,4293.67,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...
13383,72,г Тюмень,None,4,68,29,2022-06-30,Другое,51344.0,6289.98,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...
13397,72,"г Тюмень, ул Профсоюзная, д. 56",None,6,65,0,2024-12-31,Другое,44897.0,6248.51,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...
13398,72,"г Тюмень, ул Профсоюзная, д. 56",None,7,43,0,2024-12-31,Другое,47886.0,4812.98,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...
13399,72,"г Тюмень, ул Профсоюзная, д. 56",None,1,0,125,2024-12-31,Другое,50768.0,0.0,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9953 entries, 13391 to 37797
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      9953 non-null   int64  
 1   address     9927 non-null   object 
 2   name        7733 non-null   object 
 3   floors      9953 non-null   int64  
 4   flats       9953 non-null   int64  
 5   parking     9953 non-null   int64  
 6   ready_date  9953 non-null   object 
 7   material    9953 non-null   object 
 8   price       7464 non-null   float64
 9   sqare       9953 non-null   object 
 10  class       9953 non-null   object 
 11  url         9953 non-null   object 
dtypes: float64(1), int64(4), object(7)
memory usage: 1010.9+ KB


In [38]:
# сохранение в эксель 
excel_name = 'dmrf.xlsx'
with pd.ExcelWriter(excel_name) as writer:  
    df.to_excel(writer, sheet_name='sheet1')

In [39]:
# сохранение в pickle
picle_name = 'dmrf.pkl'
df.to_pickle(picle_name)

In [41]:
# сохранение в БД SQL
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:')

df.to_sql('dmrf', con=engine)

pd.read_sql('select * from dmrf', engine).head()

,id,region,address,name,floors,flats,parking,ready_date,material,price,sqare,class,url,city
0,13391,72,"г Тюмень, ул Профсоюзная",None,3,44,41,2022-06-30,Другое,55893.0,4293.67,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
1,13383,72,г Тюмень,None,4,68,29,2022-06-30,Другое,51344.0,6289.98,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
2,13397,72,"г Тюмень, ул Профсоюзная, д. 56",None,6,65,0,2024-12-31,Другое,44897.0,6248.51,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
3,13398,72,"г Тюмень, ул Профсоюзная, д. 56",None,7,43,0,2024-12-31,Другое,47886.0,4812.98,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
4,13399,72,"г Тюмень, ул Профсоюзная, д. 56",None,1,0,125,2024-12-31,Другое,50768.0,0.0,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень


## Задание 2

- Выберите регион
- Сравните среднюю цену квадратного метра в городах этого региона
- Определите количество строящихся объектов и квартир в регионах
- Определите объем строящегося жилья в квадратных метрах

In [14]:
import re

#извлечем регулярным выражением название нас.пункта

df['city'] = df['address'].str.extract(r'\bг\s*([^,]+)')

df.head()

,region,address,name,floors,flats,parking,ready_date,material,price,sqare,class,url,city
id,,,,,,,,,,,,,
13391,72,"г Тюмень, ул Профсоюзная",None,3,44,41,2022-06-30,Другое,55893.0,4293.67,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
13383,72,г Тюмень,None,4,68,29,2022-06-30,Другое,51344.0,6289.98,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
13397,72,"г Тюмень, ул Профсоюзная, д. 56",None,6,65,0,2024-12-31,Другое,44897.0,6248.51,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
13398,72,"г Тюмень, ул Профсоюзная, д. 56",None,7,43,0,2024-12-31,Другое,47886.0,4812.98,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень
13399,72,"г Тюмень, ул Профсоюзная, д. 56",None,1,0,125,2024-12-31,Другое,50768.0,0.0,Элитный,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Тюмень


In [15]:
# выберем ленобласть
df_lo = df[df['region'] == 47]
df_lo.head()

,region,address,name,floors,flats,parking,ready_date,material,price,sqare,class,url,city
id,,,,,,,,,,,,,
23682,47,д Агалатово,"Жилой комплекс ""84 высота""",10,54,0,2022-06-30,Блоки,54859.0,2748.15,Типовой,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,NaN
23494,47,"гп Янино-1, улица 7-ая линия, вл. Участок 1","ЖК ""Янинский каскад-5""",3,251,0,2022-12-31,Кирпич,38582.0,10991.85,Типовой,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,п Янино-1
19457,47,"г Сертолово, Микрорайон Сертолово-1, улица Ларина","""Авеню""",16,166,96,2023-03-31,Монолит-кирпич,61929.0,9944.7,Комфорт,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,Сертолово
41385,47,"п Бугры, корпус 22","Жилой комплекс""Энфилд""",13,451,0,2023-12-31,Монолит-кирпич,145560.0,21057.8,Комфорт,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,NaN
37725,47,"п Бугры, корпус 24","ЖК ""ЭНФИЛД""",13,176,0,2023-06-30,Монолит,128131.0,10828.62,Комфорт,https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D...,NaN


In [16]:
df_lo['city'].value_counts()

Всеволожск           18
п Янино-1            17
. Мурино             16
п Новоселье          15
ород Мурино          13
. Кудрово            11
Сертолово             7
орода Коммунар        7
Коммунар              6
Гатчина               5
Подпорожье            5
Луга                  4
Выборг                4
п Фёдоровское         4
ород Кудрово          4
Кингисепп             3
п имени Свердлова     3
п Токсово             3
п Дубровка            2
Лодейное Поле         1
п Рощино              1
Тихвин                1
Ивангород             1
Сясьстрой             1
Тосно                 1
Волхов                1
Name: city, dtype: int64

In [17]:
# регулярка сработала не на 100 процентов, попроавим вручную
df_lo.loc[df_lo['city'] == '. Мурино', 'city'] = 'Мурино'
df_lo.loc[df_lo['city'] == 'ород Мурино', 'city'] = 'Мурино'
df_lo.loc[df_lo['city'] == '. Кудрово', 'city'] = 'Кудрово'
df_lo.loc[df_lo['city'] == 'ород Кудрово', 'city'] = 'Кудрово'
df_lo.loc[df_lo['city'] == 'орода Коммунар', 'city'] = 'Коммунар'

/Users/user/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
df_lo['city'].value_counts()

Мурино               29
Всеволожск           18
п Янино-1            17
Кудрово              15
п Новоселье          15
Коммунар             13
Сертолово             7
Подпорожье            5
Гатчина               5
Луга                  4
Выборг                4
п Фёдоровское         4
Кингисепп             3
п имени Свердлова     3
п Токсово             3
п Дубровка            2
Лодейное Поле         1
Сясьстрой             1
Ивангород             1
Тихвин                1
п Рощино              1
Волхов                1
Тосно                 1
Name: city, dtype: int64

In [36]:
# Сравните среднюю цену квадратного метра в городах этого региона
df_lo.groupby('city').agg({'price':'mean'}).sort_values(by='price', ascending=False)

,price
city,
Всеволожск,136001.222222
Кудрово,123334.000000
п Янино-1,120158.411765
Гатчина,119579.800000
п Новоселье,115249.444444
Мурино,105120.210526
Сертолово,102355.428571
Выборг,101437.000000
Кингисепп,85994.666667


In [35]:
# Определите количество строящихся объектов и квартир в регионах
df_count = df_lo.groupby('city').agg({'address':'count', 'flats':'sum'}).sort_values(by='address', ascending=False)
df_count = df_count.rename(columns = {'address':'obj_count', 'flats':'flats_count'})
df_count

,obj_count,flats_count
city,,
Мурино,29,16964
Всеволожск,18,919
п Янино-1,17,3284
п Новоселье,15,6848
Кудрово,15,7558
Коммунар,13,1505
Сертолово,7,2049
Подпорожье,5,525
Гатчина,5,670


In [31]:
# Определите объем строящегося жилья в квадратных метрах
df_lo['sqare'] = df_lo['sqare'].apply(pd.to_numeric)

print('Oбъем строящегося жилья в ленобласти: ',df_lo['sqare'].sum(),'кв.м')

Oбъем строящегося жилья в ленобласти:  3201746.59 кв.м


<ipython-input-31-4bd176a43057>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lo['sqare'] = df_lo['sqare'].apply(pd.to_numeric)
